# Hierarchical Navigable Small World

Hierarchical Navigable Small World is algorithm for approximate nearest neighbors search originally presented <a href='https://arxiv.org/ftp/arxiv/papers/1603/1603.09320.pdf'>here</a>.

### Contents

* [1. Installation](#1.-Installation)
    * [1.1 Build from Arcadia for use in Arcadia](#1.1-Build-from-Arcadia-for-use-in-Arcadia)
    * [1.2 Build from Arcadia for use outside Arcadia](#1.2-Build-from-Arcadia-for-use-outside-Arcadia)
    * [1.3 Install from Yandex PyPI for use outside Arcadia](#1.3-Install-from-Yandex-PyPI-for-use-outside-Arcadia)
* [2. Data Loading](#2.-Data-Loading)
* [3. Index Building](#3.-Index-Building)
* [4. Search Index](#4.-Search-Index)
* [5. Saving and Loading Index](#5.-Saving-and-Loading-Index)
* [6. Snapshots](#6.-Snapshots)

### 1. Installation

#### 1.1 Build from Arcadia for use in Arcadia

Just add <i>/library/python/hnsw/lib</i> to your project's PEERDIR

#### 1.2 Build from Arcadia for use outside Arcadia

Go to directory <a href='https://a.yandex-team.ru/arc/trunk/arcadia/library/python/hnsw/hnsw'>/library/python/hnsw/hnsw</a> and execute next command (change python config by path to your python's config):

<i>$ ya make -DUSE_ARCADIA_PYTHON=no -DOS_SDK=local -DPYTHON_CONFIG=python-config</i>

Then from parent directory install package

<i>$ python setup.py install</i>

#### 1.3 Install from Yandex PyPI for use outside Arcadia

<i>$ pip install -i https://pypi.yandex-team.ru/simple/ hnsw</i>

### 2. Data Loading

Before work with HNSW index you should load pool of vectors from binary file with specified type and dimension of vectors.

In [1]:
from hnsw import Pool, EVectorComponentType

In [2]:
pool = Pool(vectors_path='floats_60000', dtype=EVectorComponentType.Float, dimension=10)

Pool has methods for accessing items and getting number of items in pool

In [3]:
pool.get_num_items()

6000

In [4]:
pool.get_item(123)

array([-0.08313891,  0.03384485,  0.13672869, -0.07878881, -0.05491406,
       -0.02204733,  0.02824783,  0.00267301,  0.03477978,  0.03977749],
      dtype=float32)

### 3. Index Building

Firstly create HNSW object to work with index

In [5]:
from hnsw import Hnsw, EDistance

In [6]:
index = Hnsw()

You can build index with default options by specifying only pool and distance.

In [7]:
index.build(pool, EDistance.DotProduct)


Building level 2 size 23

Building level 1 size 375

Building level 0 size 6000
Progress: 66.667%	Time passed: 1.19s
Done in 2.02s


You can customize index building by several params (see documentation for more details)

In [8]:
index.build(pool, EDistance.DotProduct, max_neighbors=200, search_neighborhood_size=1000, num_exact_candidates=300, \
            batch_size=2000, upper_level_batch_size=300, level_size_decay=2, num_threads=2)


Building level 11 size 2

Building level 10 size 5

Building level 9 size 11

Building level 8 size 23

Building level 7 size 46

Building level 6 size 93

Building level 5 size 187

Building level 4 size 375

Building level 3 size 750

Building level 2 size 1500

Building level 1 size 3000
Progress: 50.000%	Time passed: 9.44s
Building level 0 size 6000
Progress: 100.000%	ime passed: 26.5sTime passed: 36s
Done in 36s


### 4. Search Index

After building index we can search nearest neighbors. The result of search is list of tuples where first item of tuple is ID of pool item and second item of tuple is calculated distance from query to pool item. Parameter search_neighborhood_size means width of search.

In [10]:
import numpy as np

query = np.random.random(10)

index.get_nearest(query, top_size=10, search_neighborhood_size=100)

[(2123L, 0.5717909932136536),
 (4883L, 0.5507800579071045),
 (5003L, 0.5399731993675232),
 (3743L, 0.5265855193138123),
 (863L, 0.5257172584533691),
 (5183L, 0.5169099569320679),
 (3323L, 0.5118076205253601),
 (2483L, 0.4830664396286011),
 (3143L, 0.4694557785987854),
 (623L, 0.4500829875469208)]

There is also another parameter of search: distance_calc_limit. It can be useful if you want to limit search steps in index to reduce search execution time with small decrease of accuracy.

### 5. Saving and Loading Index

One of typical cases of usage HNSW library is to build index, save it and use later. For this case there are functions for saving and loading index.

In [12]:
index.save('hnsw_index')

In [13]:
index = Hnsw()
index.load('hnsw_index', pool, EDistance.DotProduct)

In [14]:
index.get_nearest(query, top_size=5, search_neighborhood_size=10)

[(5465L, 0.38982123136520386),
 (4721L, 0.3159331977367401),
 (2123L, 0.3116426467895508),
 (3323L, 0.30608993768692017),
 (2407L, 0.30178695917129517)]

### 6. Snapshots

If index building lasts a long time it is recommended to save snapshots sometimes. If something goes wrong and building has crashed (for example, because of memory limit exceeding), you can continue building from last saved snapshot. You can check snapshots by interrupting execution of next cell and executing it again.

In [21]:
index = Hnsw()
index.build(pool, EDistance.DotProduct, max_neighbors=200, search_neighborhood_size=1000, num_exact_candidates=300, \
            batch_size=2000, upper_level_batch_size=300, level_size_decay=2, num_threads=2, \
            snapshot_file='snapshot', snapshot_interval=1)

Restored from snapshot

Building level 1 size 3000
Progress: 50.000%	Time passed: 8.73s
Snapshot saved to snapshot

Snapshot saved to snapshot

Building level 0 size 6000
Progress: 83.333%	Time passed: 26.7s
Snapshot saved to snapshot
Progress: 100.000%	Time passed: 36.8s
Snapshot saved to snapshot

Snapshot saved to snapshot

Done in 37.4s
